# ECS529U Algorithms and Data Structures
# Lab sheet 9

This lab gets you to work with hash tables. Some helper code has been provided for you in `pab9-prep.ipynb`, including a pretty-printing function `__str__` for `HashTable`.

**Marks (max 5):**  Question 1: 1.5 | Questions 3-5: 0.5 each | Questions 2,6: 1 each

## Question 0 [no marks]

In some of the following questions you will be asked to provide an answer which depends on the following numbers. Please calculate the numbers and array below:

- `SID`: this is your QMUL student number. It should be a 9-digit number.
- `SID2`: this is `SID` squared. It should be a 17-digit number.
- $u_0$, $u_1$, $u_2$, $u_3$, $u_4$, $u_5$, $u_6$, $u_7$, $u_8$, $u_{9}$: these are the first 10 __unique__ digits  (between 0 and 9) in the sequence `SID2`, from left to right. If there are less than 10 unique
digits in your `SID2`, you should fill in your sequence of $u$'s
with the remaining digits in increasing order. 
In the end, your sequence
$u_0, u_1, \dots, u_9$
should
contain all digits between 0 and 9 (in some order).

Let `U` be the array [$u_0, u_1, u_2, u_3, u_4, u_5, u_6, u_7, u_8, u_{9}$]. __Write down the array `U`.__

For example, if your student ID is 200435842, then its square is 40174526758248964.
This gives us the following unique digits, from left to right:

    4,0,1,7,5,2,6,8,9
    
To get the sequence of $u$'s we simply need to fill in the number 3 at the end. Thus:

    U = [4,0,1,7,5,2,6,8,9,3]

## Question 1 [1.5]

Consider hash tables with underlying array of size 7, and (initial) hash function the remainder of dividing by 7:

    def hash(d): return i%7
    
Let `A` be the array `[412, 21, 420, 3, 56, 99, 1528, 21, 462, 409]` and calculate the array:

    B = [A[U[i]] for i in range(10)]

a) Draw the hash table we obtain if we start from the empty one and add consecutively the elements of `B`, 
without resizing up the underlying array.

b) Draw the hash table we obtain if we start from the empty one and add consecutively the elements of `B`, but this time using resizing of the underlying array whenever the load factor exceeds 0.75. 

The hash function should be updated accordingly after each resize operation. You can use Python to calculate remainder operations.

In [15]:
class Node:
    def __init__(self, d, n):
        self.data = d
        self.next = n

class LinkedList:
    def __init__(self):
        self.head = None
        self.length = 0

    def __str__(self):
        if self.head == None: 
            return "empty"
        st = "-"
        ptr = self.head
        while ptr != None:
            st += "-> "+str(ptr.data)+" "
            ptr = ptr.next
        return st+"|"
    
    def search(self, d):
        i = 0
        ptr = self.head
        while ptr != None:
            if ptr.data == d:
                return i
            ptr = ptr.next
            i += 1
        return -1
        
    def append(self, d):
        if self.head == None:      
            self.head = Node(d,None) 
        else:
            ptr = self.head
            while ptr.next != None:
                ptr = ptr.next
            ptr.next = Node(d,None)
        self.length += 1

    def insert(self, i, d):
        if self.head == None or i == 0:
            self.head = Node(d,self.head)
        else:
            ptr = self.head
            while i>1 and ptr.next != None:
                ptr = ptr.next
                i -= 1
            ptr.next = Node(d,ptr.next)
        self.length += 1

    def remove(self, i): # removes i-th element and returns it
        if self.head == None:
            return None
        if i == 0:
            val = self.head.data
            self.head = self.head.next
            self.length -= 1
            return val
        else:
            ptr = self.head
            while i>1 and ptr.next != None:
                ptr = ptr.next
                i -= 1
            if i == 1:
                val = ptr.next.data
                ptr.next = ptr.next.next
                self.length -= 1
                return val
            return None
    
    def removeVal(self, d):
        if self.head == None:
            return
        if self.head.data == d:
            self.head = self.head.next
            self.length -= 1
        else:
            ptr = self.head
            while ptr.next != None:
                if ptr.next.data == d:
                    ptr.next = ptr.next.next
                    self.length -= 1
                    break
                ptr = ptr.next

In [10]:
class HashTable:
    def __init__(self, d=10):    # a default array length of 10
        self.inArray = [LinkedList() for i in range(d)] 
        self.size = 0
        self.threshold = 0.75    # a default threshold of 0.75

    def hash(self, d):
        return d % len(self.inArray)
           
    def add(self, d):
        i = self.hash(d)
        self.inArray[i].insert(0,d)
        self.size += 1
        # condition that must be respected: self.size <= self.threshold * len(self.inArray)
        if self.size > self.threshold*len(self.inArray): self._resizeUp()
        
    def search(self, d):
        i = self.hash(d)
        if self.inArray[i].search(d) == -1: return False
        return True

    def remove(self, d):
        i = self.hash(d)
        oldLength = self.inArray[i].length
        self.inArray[i].removeVal(d)
        if self.inArray[i].length < oldLength:
            self.size -= 1

    def _resizeUp(self):
        oldArray = self.inArray
        self.inArray = [LinkedList() for _ in range(2*len(oldArray))]
        for l in oldArray:
            while l.length > 0:
                d = l.remove(0)
                self.inArray[self.hash(d)].insert(0,d)            
            
    # these are to make our life easier in adding/removing many elements
    def addAll(self, A):
        for i in range(len(A)):
            self.add(A[i])

    def removeAll(self, A):
        for i in range(len(A)):
            self.remove(A[i])

            
def myprint(h):
    for i in range(len(h.inArray)):
        print("pos",i,":",h.inArray[i])
    print()
        



In [11]:
h = HashTable(10)
print("hashtable (size "+str(h.size)+"):\n"); myprint(h)
input()
h.addAll([0, 34, 23, 4, 5, -2, -13]) # add 7 elements
print("hashtable (size "+str(h.size)+"):\n"); myprint(h)
input()
h.add(42) # add 1 more, so 8 elements now > 0.75*10 -- need to resize up
print("hashtable (size "+str(h.size)+"):\n"); myprint(h)
input()
h.addAll([
]) # add a few more
print("hashtable (size "+str(h.size)+"):\n"); myprint(h)
input()
h.removeAll([0, 34, 23, 4, 5, 12, 45, 42, 45, 10, 56, 90, 99, 10, 2, 2, 2103]) # and now remove them
print("hashtable (size "+str(h.size)+"):\n"); myprint(h)

hashtable (size 0):

pos 0 : empty
pos 1 : empty
pos 2 : empty
pos 3 : empty
pos 4 : empty
pos 5 : empty
pos 6 : empty
pos 7 : empty
pos 8 : empty
pos 9 : empty
hashtable (size 7):

pos 0 : --> 0 |
pos 1 : empty
pos 2 : empty
pos 3 : --> 23 |
pos 4 : --> 4 -> 34 |
pos 5 : --> 5 |
pos 6 : empty
pos 7 : --> -13 |
pos 8 : --> -2 |
pos 9 : empty
hashtable (size 8):

pos 0 : --> 0 |
pos 1 : empty
pos 2 : --> 42 |
pos 3 : --> 23 |
pos 4 : --> 4 |
pos 5 : --> 5 |
pos 6 : empty
pos 7 : --> -13 |
pos 8 : empty
pos 9 : empty
pos 10 : empty
pos 11 : empty
pos 12 : empty
pos 13 : empty
pos 14 : --> 34 |
pos 15 : empty
pos 16 : empty
pos 17 : empty
pos 18 : --> -2 |
pos 19 : empty
hashtable (size 8):

pos 0 : --> 0 |
pos 1 : empty
pos 2 : --> 42 |
pos 3 : --> 23 |
pos 4 : --> 4 |
pos 5 : --> 5 |
pos 6 : empty
pos 7 : --> -13 |
pos 8 : empty
pos 9 : empty
pos 10 : empty
pos 11 : empty
pos 12 : empty
pos 13 : empty
pos 14 : --> 34 |
pos 15 : empty
pos 16 : empty
pos 17 : empty
pos 18 : --> -2 |
pos 1

In [8]:
Initial Hash Table:
[_, _, _, _, _, _, _]

Add 412:
[_, _, _, _, _, _, 412]

Add 21:
[_, 21, _, _, _, _, 412]

Add 420:
[_, 21, _, _, 420, _, 412]

Add 3:
[3, 21, _, _, 420, _, 412]

Add 56:
[3, 21, _, _, 420, 56, 412]

Add 99:
[3, 21, _, 99, 420, 56, 412]

Add 1528:
[3, 21, _, 99, 420, 56, 412, 1528]

Add 21 (collision):
[3, 21, _, 99, 420, 56, 412, 1528]

Add 462:
[3, 21, 462, 99, 420, 56, 412, 1528]

Add 409:
[3, 21, 462, 99, 420, 56, 412, 1528, 409]



Initial Hash Table:
[_, _, _, _, _, _, _]

Add 412:
[_, _, _, _, _, _, 412]

Add 21:
[_, 21, _, _, _, _, 412]

Add 420:
[_, 21, _, _, 420, _, 412]

Load factor exceeds 0.75, resize to size 14:
[_, _, _, _, _, _, _, _, _, _, _, _, _, _]

Rehash and Add 3:
[3, _, _, _, _, _, _, _, _, _, _, _, _, _]

Rehash and Add 56:
[3, _, _, _, _, _, _, _, _, _, _, _, _, 56]

Rehash and Add 99:
[3, _, _, _, _, _, _, 99, _, _, _, _, _, 56]

Rehash and Add 1528:
[3, _, _, _, _, _, _, 99, _, _, _, _, _, 56, 1528]

Load factor exceeds 0.75, resize to size 28:
[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _,



SyntaxError: invalid syntax (4248892659.py, line 1)

## Question 2 [1]

Add in `HashTable` (the code of which you can find in `lecture9.ipynb`) the following functions:

    a) def max(self)

that returns the largest element in the hashtable (represented by `self`). Your solution should not use sorting.

    b) def toArray(self)

that returns an array containing all elements in the hashtable (represented by `self`),
in no particular order. For example, if `h` is a hashtable constructed by:

    h = HashTable(); h.addAll([5,10,0,42,42])
    
then `h.toArray()` should return an array that contains the same elements as the array `[5,10,0,42,42]`.

    c) def count(self, d)
    
that returns the number of times that value `d` occurs in the hash table (represented by `self`). Note your function should not change the hash table. For example, the following code:

    h = HashTable(); h.addAll([5,10,0,42,42])
    print(h.count(0),h.count(1),h.count(42))
    
should print `1 0 2`. 

*Hint:* you can add a corresponding function `count` in `LinkedList` and simply return the result of counting for `d` in the linked list in position `hash(d)` of the underlying array.

In [12]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, value):
        new_node = Node(key, value)
        if not self.head:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    # Add a count method to LinkedList
    def count(self, d):
        count = 0
        current = self.head
        while current:
            if current.value == d:
                count += 1
            current = current.next
        return count

class HashTable:
    def __init__(self, size):
        self.size = size
        self.array = [LinkedList() for _ in range(size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def add(self, key, value):
        index = self.hash_function(key)
        self.array[index].append(key, value)

    # Add a max method to HashTable
    def max(self):
        max_value = float('-inf')
        for linked_list in self.array:
            current = linked_list.head
            while current:
                max_value = max(max_value, current.value)
                current = current.next
        return max_value

    # Add a toArray method to HashTable
    def toArray(self):
        result = []
        for linked_list in self.array:
            current = linked_list.head
            while current:
                result.append(current.value)
                current = current.next
        return result

    # Add a count method to HashTable
    def count(self, d):
        count = 0
        index = self.hash_function(d)
        count += self.array[index].count(d)
        return count

# Your code
h = HashTable(10)
print("hashtable (size " + str(h.size) + "):\n"); myprint(h)
input()
h.addAll([0, 34, 23, 4, 5, -2, -13])  # add 7 elements
print("hashtable (size " + str(h.size) + "):\n"); myprint(h)
input()
h.add(42)  # add 1 more, so 8 elements now > 0.75*10 -- need to resize up
print("hashtable (size " + str(h.size) + "):\n"); myprint(h)
input()
h.addAll([
])  # add a few more
print("hashtable (size " + str(h.size) + "):\n"); myprint(h)
input()
h.removeAll([0, 34, 23, 4, 5, 12, 45, 42, 45, 10, 56, 90, 99, 10, 2, 2, 2103])  # and now remove them
print("hashtable (size " + str(h.size) + "):\n"); myprint(h)

# Test the new functions
print("Max value:", h.max())
print("Array representation:", h.toArray())
print("Count occurrences:", h.count(0), h.count(1), h.count(42))


hashtable (size 10):


AttributeError: 'HashTable' object has no attribute 'inArray'

## Question 3 [0.5]

Add in `HashTable` a function:

    def __str__(self)

that returns a string representing the hashtable in the following format. 

- The string should contain a sequence of lines, one for each entry of the underlying array, separated by newlines ('\n'). 
- Each of these lines should start with the index of the underlying array, included in square brackets, followed by a string representation of the linked list in that index.

For example, if `h` is a hashtable with an underlying array of size 7 constructed by:

    h = HashTable(7); h.addAll([42,2049,12,5,18])
    
then, `str(h)` should return the string:

    [0] -> 42\n[1]\n[2]\n[3]\n[4] -> 18\n[5] -> 5 -> 12 -> 2049\n[6]\n'

And, therefore, `print(h)` would print:

    [0] -> 42
    [1]
    [2]
    [3]
    [4] -> 18
    [5] -> 5 -> 12 -> 2049
    [6]
    
__Hint:__ You might find useful to replace the `__str__` function of `LinkedList` with one that is more suitable for this task (see `lab9-prep.ipynb`).

In [13]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None
class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, value):
        new_node = Node(key, value)
        if not self.head:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    # Add a count method to LinkedList
    def count(self, d):
        count = 0
        current = self.head
        while current:
            if current.value == d:
                count += 1
            current = current.next
        return count

    # Modify the __str__ method for a more suitable representation
    def __str__(self):
        result = []
        current = self.head
        while current:
            result.append(str(current.value))
            current = current.next
        return ' -> '.join(result)

class HashTable:
    def __init__(self, size):
        self.size = size
        self.array = [LinkedList() for _ in range(size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def add(self, key, value):
        index = self.hash_function(key)
        self.array[index].append(key, value)

    # Add a max method to HashTable
    def max(self):
        max_value = float('-inf')
        for linked_list in self.array:
            current = linked_list.head
            while current:
                max_value = max(max_value, current.value)
                current = current.next
        return max_value

    # Add a toArray method to HashTable
    def toArray(self):
        result = []
        for linked_list in self.array:
            current = linked_list.head
            while current:
                result.append(current.value)
                current = current.next
        return result

    # Add a count method to HashTable
    def count(self, d):
        count = 0
        index = self.hash_function(d)
        count += self.array[index].count(d)
        return count

    # Modify the __str__ method for a more suitable representation
    def __str__(self):
        result = []
        for i, linked_list in enumerate(self.array):
            result.append(f'[{i}] -> {str(linked_list)}')
        return '\n'.join(result)


h = HashTable(7)
h.addAll([42, 2049, 12, 5, 18])


print(str(h))


AttributeError: 'HashTable' object has no attribute 'addAll'

## Question 4 [0.5]

Re-implement the `HashTable` function

    def hash(self, d)

so that, for each input integer `d`, it adds together all the digits of `d` and takes the remainder 
of dividing the resulting number by the length of the internal array. For example, if `h` is built 
as follows:

    h = HashTable(7); h.addAll([42,2049,12,5,18])
    
then `print(h)` should print (using your implementation of `str`):

    [0]
    [1] -> 2049
    [2] -> 18
    [3] -> 12
    [4]
    [5] -> 5
    [6] -> 42

## Question 5 [0.5]

Re-implement the `HashTable` function

    def hash(self, d)

so that it works for `d` being an array of integers. For each input array `d`, it adds together the elements of `d` and takes the remainder 
of dividing the resulting number by the length of the internal array. For example, if `h` is built 
as follows:

    h = HashTable(7); h.addAll([[],[1,2,3],[70,7,77,14],[42,2049,12,5,18]])
    
then `print(h)` should print (using your implementation of `str`):

    [0] -> [70, 7, 77, 14] -> []
    [1]
    [2]
    [3]
    [4]
    [5] -> [42, 2049, 12, 5, 18]
    [6] -> [1, 2, 3]

In [14]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, value):
        new_node = Node(key, value)
        if not self.head:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    # Add a count method to LinkedList
    def count(self, d):
        count = 0
        current = self.head
        while current:
            if current.value == d:
                count += 1
            current = current.next
        return count

    # Modify the __str__ method for a more suitable representation
    def __str__(self):
        result = []
        current = self.head
        while current:
            result.append(str(current.value))
            current = current.next
        return ' -> '.join(result)

class HashTable:
    def __init__(self, size):
        self.size = size
        self.array = [LinkedList() for _ in range(size)]

    # Re-implement the hash function for single integers
    def hash(self, d):
        return sum(int(digit) for digit in str(d)) % self.size

    def add(self, key, value):
        index = self.hash(key)
        self.array[index].append(key, value)

    # Add a max method to HashTable
    def max(self):
        max_value = float('-inf')
        for linked_list in self.array:
            current = linked_list.head
            while current:
                max_value = max(max_value, current.value)
                current = current.next
        return max_value

    # Add a toArray method to HashTable
    def toArray(self):
        result = []
        for linked_list in self.array:
            current = linked_list.head
            while current:
                result.append(current.value)
                current = current.next
        return result

    # Add a count method to HashTable
    def count(self, d):
        count = 0
        index = self.hash(d)
        count += self.array[index].count(d)
        return count

    # Modify the __str__ method for a more suitable representation
    def __str__(self):
        result = []
        for i, linked_list in enumerate(self.array):
            result.append(f'[{i}] -> {str(linked_list)}')
        return '\n'.join(result)

# Your code
h1 = HashTable(7)
h1.addAll([42, 2049, 12, 5, 18])

h2 = HashTable(7)
h2.addAll([[], [1, 2, 3], [70, 7, 77, 14], [42, 2049, 12, 5, 18]])

# Test the __str__ method for both cases
print("Hash Table 1:")
print(str(h1))

print("\nHash Table 2:")
print(str(h2))


AttributeError: 'HashTable' object has no attribute 'addAll'

## Hash Maps

In the last question we will implement hash maps, which are hash tables that store objects 
that are pairs of the form: (key,value). Each key in the hash map has at most one 
corresponding value. Hashing of such pairs (in order to search for them, remove them, etc.) is done with respect
to their key. That is, to search if there is a pair with key 42, we need to hash 42 to retrieve 
an index, and then search the linked list in the corresponding entry of the internal array for 
a pair with key 42. 

To store pairs, we will use the following class, where note that we take two pairs to be 
equal just if their keys are equal. 

    class KVPair:
        def __init__(self, k, v):
            self.key = k
            self.val = v
        def __eq__(self, other):
            return self.key == other.key

## Question 6 [1]

Adapt the implementation of `HashTable` into a class `HashMap` with functions:

- `def search(self,k)` : searches for a pair with key `k` in the hash map and returns 
`True` if it finds one, otherwise `False`.
- `def remove(self,k)` : searches for a pair with key `k` in the hash map and removes 
it if it finds its, otherwise it leaves the hash map unchanged.
- `def get(self,k)` : returns the value `v` with which the key `k` is paired in the hash 
map (i.e. such that `(k,v)` is in the hash map). If there is no such pair, returns `None`.
- `def put(self,k,v)` : adds the pair `(k,v)` in the hash map. If there is already a pair 
with key `k` in the map, then its value is updated to `v`.

Your hash maps should work with keys and values of arbitrary type (so, you would need to  use a hash function like the one in Question 5). 

To help you with the implementation, we have made below and implemented the constructor and the `search` function. In the same file, we have 
included an implementation of `LinkedList` with a couple of additional functions that can be
of help (`toSeqString`, for printing a linked list as a comma-separated sequence, and `get`).

For example, executing the following code:

    h = HashMap()
    h.put(10,12); print(h, h.size, h.search(10), h.search(42))
    h.put(10,13); h.put(42,12); print(h, h.size, h.search(10), h.search(42))
    h.put("Algorithms","Are Great!"); print(h)

we should get the printout:

    {(10, 12)} 1 True False
    {(10, 13),(42, 12)} 2 True True
    {(10, 13),(42, 12),(Algorithms, Are Great!)}

In [ ]:
class KVPair:
    def __init__(self, k, v):
        self.key = k
        self.val = v

    def __eq__(self, other):
        return self.key == other.key

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, value):
        new_node = Node(KVPair(key, value))
        if not self.head:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def search(self, key):
        current = self.head
        while current:
            if current.value.key == key:
                return True
            current = current.next
        return False

    def remove(self, key):
        if not self.head:
            return
        if self.head.value.key == key:
            self.head = self.head.next
            return
        current = self.head
        while current.next:
            if current.next.value.key == key:
                current.next = current.next.next
                return
            current = current.next

    def get(self, key):
        current = self.head
        while current:
            if current.value.key == key:
                return current.value.val
            current = current.next
        return None

    def toSeqString(self):
        result = []
        current = self.head
        while current:
            result.append(f'({current.value.key}, {current.value.val})')
            current = current.next
        return ', '.join(result)

class HashMap:
    def __init__(self, size=10):
        self.size = size
        self.array = [LinkedList() for _ in range(size)]

    def hash(self, key):
        return sum(int(digit) for digit in str(key)) % self.size

    def search(self, key):
        index = self.hash(key)
        return self.array[index].search(key)

    def remove(self, key):
        index = self.hash(key)
        self.array[index].remove(key)

    def get(self, key):
        index = self.hash(key)
        return self.array[index].get(key)

    def put(self, key, value):
        index = self.hash(key)
        linked_list = self.array[index]
        if linked_list.search(key):
            linked_list.remove(key)
        linked_list.append(key, value)

    def __str__(self):
        result = "{"
        for linked_list in self.array:
            result += linked_list.toSeqString() + ', '
        result = result.rstrip(', ') + "}"
        return result

# Test the HashMap class
h = HashMap()
h.put(10, 12)
print(h, h.size, h.search(10), h.search(42))

h.put(10, 13)
h.put(42, 12)
print(h, h.size, h.search(10), h.search(42))

h.put("Algorithms", "Are Great!")
print(h)
